# Apache Airflow для аналитика

# Решение примера на Airflow: практика

## Настройка Airflow

In [1]:
# Установка Airflow

#!pip install apache-airflow==2.1.4
#!pip install wtforms==2.3.3

# Инициализация базы данных

#!airflow db init

In [2]:
# Создадим необходимые папки

#!mkdir /root/airflow/dags
#!touch /root/airflow/dags/dag.py

In [3]:
# Включим веб-сервер

#!airflow webserver -p 18273 -D

In [4]:
# Последующие команды не имеют отношения к Airflow
# Они нужны только для корректной работы веб морды
# в среде Google Colab

#!pip install pyngrok
#!ngrok authtoken <> # найти его можно https://dashboard.ngrok.com/get-started/setup

In [5]:
# Создадим пользователя Airflow

#!airflow users create \
#          --username admin \
#          --firstname admin \
#          --lastname admin \
#          --role Admin \
#          --email admin@example.org \
#          -p 12345

Поместим в dag.py следующий код:

In [6]:
#from airflow import DAG
#from datetime import timedelta
#from airflow.utils.dates import days_ago
#from airflow.operators.dummy_operator import DummyOperator

#dag = DAG('dag',schedule_interval=timedelta(days=1), start_date=days_ago(1))
#t1 = DummyOperator(task_id='task_1', dag=dag)
#t2 = DummyOperator(task_id='task_2',dag=dag)
#t3 = DummyOperator(task_id='task_3',dag=dag)
#t4 = DummyOperator(task_id='task_4',dag=dag)
#t5 = DummyOperator(task_id='task_5',dag=dag)
#t6 = DummyOperator(task_id='task_6',dag=dag)
#t7 = DummyOperator(task_id='task_7',dag=dag)

In [7]:
# Запуск шедулера

#!airflow scheduler -D

In [8]:
# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/endpoints
# При каждом отключении ссылка будет меняться

#!nohup ngrok http 18273 > /dev/null &

После запуска команды выше, перейдите по адресу в ngrok и подождите пока появится DAG с именем dag

## Задача

Указать верные trigger_rule чтобы дочерние задачки отработали. Дочерние задачи это task_3, task_7. 

In [10]:
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.python import PythonOperator
from airflow.exceptions import AirflowSkipException, AirflowFailException

dag = DAG('dag',
          schedule_interval=timedelta(days=1),
          start_date=days_ago(1))


# Функция которая всегда верна
def success():
    pass

# Функция которая скипает задачу
def skip():
    raise AirflowSkipException

# Функция которая падает с ошибкой
def failed():
    raise AirflowFailException

task_0 = PythonOperator(
  task_id='task_0',
  python_callable=success,
  dag=dag
)

task_1 = PythonOperator(
  task_id='task_1',
  python_callable=skip,
  dag=dag
)

task_2 = PythonOperator(
  task_id='task_2',
  python_callable=failed,
  dag=dag
)

task_3 = PythonOperator(
  task_id='task_3',
  trigger_rule='all_done',
  python_callable=lambda: print("Success"),
  dag=dag
)

[task_0, task_1, task_2] >> task_3



task_4 = PythonOperator(
  task_id='task_4',
  python_callable=success,
  dag=dag
)

task_5 = PythonOperator(
  task_id='task_5',
  python_callable=failed,
  dag=dag
)

task_6 = PythonOperator(
  task_id='task_6',
  python_callable=failed,
  dag=dag
)

task_7 = PythonOperator(
  task_id='task_7',
  python_callable=lambda: print("Success"),
  trigger_rule='one_failed',
  dag=dag
)

[task_4, task_5,task_6] >> task_7

Google Collab: https://colab.research.google.com/drive/1zAtUmDoIEI9GX7SaXuehMRUCW5aHnpYU


_____________________________________________________________________________________________________________________________
Проектную работу выполнила: аналитик данных Малахова Наталья

Мой телеграм-канал: [Дневник аналитика](https://t.me/diary_musician_analyst "Дневник аналитика")

GitHub: [GitHub](https://github.com/Malakhova-Natalya "GitHub")

Habr: [Habr](https://habr.com/ru/users/Malakhova-Natalya/publications/articles/ "Habr")

Спасибо за внимание!